In [ ]:
%matplotlib inline
import random, pylab
import numpy as np
import numpy
import matplotlib.pyplot as plt
import random

In [ ]:
#padding on the y axis between label and axis
plt.rcParams['ytick.major.pad']='10'

dpi_out = 400
fig_width = 3.39
golden_mean = (np.sqrt(5)-1.0)/2.0    # Aesthetic ratio
fig_height = fig_width*golden_mean # height in inches
MAX_HEIGHT_INCHES = 8.0
if fig_height > MAX_HEIGHT_INCHES:
    print("WARNING: fig_height too large:" + fig_height + 
          "so will reduce to" + MAX_HEIGHT_INCHES + "inches.")
    fig_height = MAX_HEIGHT_INCHES
#fig, ax = plt.subplots(figsize = (fig_width, fig_height), dpi=400, frameon=True)
small_tick_size = 8
small_label_size = 8

# Tutorial 1: Exponential convergence of the Markov chain algotithm

## MCMC dynamics on the $3 \times 3$ lattice

In this tutorial, we will be analyzing in some detail the Markov chain
Monte Carlo algorithm applied to the $3 \times 3$ lattice. The system
is described with lattice cells index between $0$ and $9$, and unless otherwise stated, transition probabilities are uniform. Moreover, rejections are handled as explained in the lecture.

### Questions:

1. Assuming the position at time $t$ is $r(t) = 8$ what are the possible values for $r(t+1)$?.

2. Assuming $r(0) = 8$, what is the minimum value $t_0$ of $t$ such that $P[r(t_0) = 0]  > 0$ ?

3. Evaluate $P[ r(4) = 0 | r(0) = 8]$.

## Numerical simulation

The following simulates the MCMC algorithm with a start on site number 8.

Take some time to understand it and try it out. In particular, determine which direction corresponds to which integer between 0 and 3. You can change the initial site, and also make the simulation longer in order to see more of the dynamics.

In [ ]:
neighbor =  [[1, 3, 0, 0], [2, 4, 0, 1], [2, 5, 1, 2],
             [4, 6, 3, 0], [5, 7, 3, 1], [5, 8, 4, 2],
             [7, 6, 6, 3], [8, 7, 6, 4], [8, 8, 7, 5]]
t_max = 4
site = 8
t = 0
print site
while t < t_max:
    t += 1
    site = neighbor[site][random.randint(0, 3)]
    print site

The following simulates the MCMC algorithm with a start on site number 8, and a graphical output.

Take some time to understand it and try it out. If run long enough, this would lead to homogeneous sampling of the grid.

In [ ]:
sigma = 0.4  # sigma and s_map are needed for the graphical output
s_map = [(1.0, 1.0), (2.0, 1.0), (3.0, 1.0), 
         (1.0, 2.0), (2.0, 2.0), (3.0, 2.0), 
         (1.0, 3.0), (2.0, 3.0), (3.0, 3.0)] 
neighbor =  [[1, 3, 0, 0], [2, 4, 0, 1], [2, 5, 1, 2],
             [4, 6, 3, 0], [5, 7, 3, 1], [5, 8, 4, 2],
             [7, 6, 6, 3], [8, 7, 6, 4], [8, 8, 7, 5]]
site = 8
N_runs = 10
for run in range(N_runs):
    if run < 10:
        number_string = '0' + str(run)
    else:
        number_string = str(run)

    # Begin graphical output
    cir = pylab.Circle(s_map[site], radius=sigma, fc='r')
    pylab.gca().add_patch(cir)
    pylab.plot([0.5, 3.5], [1.5, 1.5], 'b')
    pylab.plot([0.5, 3.5], [2.5, 2.5], 'b')
    pylab.plot([1.5, 1.5], [0.5, 3.5], 'b')
    pylab.plot([2.5, 2.5], [0.5, 3.5], 'b')
    pylab.title('t = '+ number_string)
    pylab.axis('scaled')
    pylab.axis([0.5, 3.5, 0.5, 3.5])
    pylab.xticks([])
    pylab.yticks([])
    pylab.savefig('pebble_basic_movie_' + number_string + '.png',
                  transparent=False)
    pylab.show()
    pylab.clf()
    # End of graphical output
    site = neighbor[site][ random.randint(0, 3)]

## Study of the convergence to the steady state

Instead of one very long run, we will now proceed with a number of shorter runs, each of which will start in the upper right corner. Numerically, this is simply achieved by adding a loop, executed N_runs times, which calls the code presented in the preceding cell.

In the following code, the stopping time is held by the variable "run". Note the new numbering scheme of the sites, which runs from 1 to 9. Histograms of the final position of the Markov chain are displayed, for the given value of the stopping time.

* Take a look at the code, and understand its structure.
* Run the code for final times 1, 2, 3, and comment on the output.

In [ ]:
xvec = {1:3, 2:2, 3:1, 4:3, 5:2, 6:1, 7:3, 8:2, 9:1} 
yvec = {1:1, 2:1, 3:1, 4:2, 5:2, 6:2, 7:3, 8:3, 9:3} 

neighbor = {1 : [2, 4, 1, 1], 2 : [3, 5, 1, 2], 3 : [3, 6, 2, 3],
            4 : [5, 7, 4, 1], 5 : [6, 8, 4, 2], 6 : [6, 9, 5, 3],
            7 : [8, 7, 7, 4], 8 : [9, 8, 7, 5], 9 : [9, 9, 8, 6]}

list_vec = []

run = 0
N_runs = 10000
if run < 10:
    run_str= '0' + str(run)
else:
    run_str = str(run)
for run_index in range(N_runs): 
    pos = 9
    for iter in range(run):
        pos = neighbor[pos][random.randint(0, 3)]
    list_vec.append(pos)

x = [xvec[k] for k in list_vec]
y = [yvec[k] for k in list_vec]

plt.xticks([])
plt.yticks([])
H, xedges, yedges = np.histogram2d(x, y, bins=(3, 3), 
           range=[[1,3],[1,3]], normed=True)

H /= np.sum(H)
print H
extent = [yedges[0], yedges[-1], xedges[-1], xedges[0]]
plt.imshow(H, extent=extent, interpolation='nearest', vmin=0.0, vmax=1.0)
plt.set_cmap('hot')
plt.colorbar()
plt.title('t = '+str(run),fontsize=22)
plt.savefig('3x3_pebble_run_'+run_str+'.png')
plt.show()


### Questions

* Run the code for final times 1, 2, 3, and comment on the output.
* Using a stopping time of 10, and N_runs = 100, and observe the final value in the lower left cell of the output table. How can you interpret that number in words?
* Is the probability of being at the starting site always strictly greater than the probability of being at any other site, for any $t>0$, as is easily checked, sor starting site 9, and t = 0, 1, etc...?
* Would our results change if, instead of running the simulation consecutively N_runs times, we let N_runs pebbles move simultaneously through the lattice?

## Analytical approach: Transfer matrices

For relatively small systems, the following approach may be followed. It is quite straightforward to evaluate the exact probabilities for the MCMC to end on each lattice site for $t=0$, $t=1$, or $t=2$. For larger times, we introduce a vector of probabilities:
$\pi^t = \begin{pmatrix} \pi_0^t \\ \pi_1^t \\ \vdots \\ \pi_8^t \end{pmatrix}$.

If we view the MCMC algorithm as the bare set of transition probabilities from one configuration to the other, then it is natural to represent it by a matrix, called transfer matrix. In our case, it is a $9 \times 9$ matrix, because there are nine different configurations in the system:

$\begin{pmatrix} p(0 \to 0) & p(1 \to 0) & p(2 \to 0) & p(3 \to 0) & p(4 \to 0) & p(5 \to 0) & p(6 \to 0) & p(7 \to 0) & p(8 \to 0) \\
p(0 \to 1) & p(1 \to 1) & p(2 \to 1) & p(3 \to 1) & p(4 \to 1) & p(5 \to 1) & p(6 \to 1) & p(7 \to 1) & p(8 \to 1) \\
p(0 \to 2) & p(1 \to 2) & p(2 \to 2) & p(3 \to 2) & p(4 \to 2) & p(5 \to 2) & p(6 \to 2) & p(7 \to 2) & p(8 \to 2) \\
\vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots & \vdots \\
p(0 \to 8) & p(1 \to 8) & p(2 \to 8) & p(3 \to 8) & p(4 \to 8) & p(5 \to 8) & p(6 \to 8) & p(7 \to 8) & p(8 \to 8) \\
\end{pmatrix}$

We may evaluate each matrix element for our particular system, leading to the following transfer matrix:

$T = \dfrac{1}{4}
\begin{pmatrix}
2 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
1 & 1 & 1 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 1 & 2 & 0 & 0 & 1 & 0 & 0 & 0 \\
1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 \\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 \\
0 & 0 & 0 & 1 & 0 & 0 & 2 & 1 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 1 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 2 \\
\end{pmatrix}$

and then use the relation between the probability vectors at time $t$ and $t+1$, and the transfer matrix: $\pi^{t+1} = T \pi^{t}$.

The following code implements this relation in order to compute and display the ${\bf exact}$ probabilities for the Markov chain to end at each site.

In [ ]:
neighbor =  [[1, 3, 0, 0], [2, 4, 0, 1], [2, 5, 1, 2],
             [4, 6, 3, 0], [5, 7, 3, 1], [5, 8, 4, 2],
             [7, 6, 6, 3], [8, 7, 6, 4], [8, 8, 7, 5]]

transfer = numpy.zeros((9, 9))

for k in range(9):
    for neigh in range(4):
        transfer[neighbor[k][neigh], k] += 0.25

position = numpy.zeros(9)
position[8] = 1.0

for t in range(100):
    print t, '  ', ["%0.5f" % i for i in position]
    position = numpy.dot(transfer, position)

* Observe the profile of P[r(4) = 0 | r(0) = 8], and confirm your previous evaluation.
* Observe the convergence to the steady state value of $1 / 9$: Application of the transfer matrix to the vector does not modify the vector: the equilibrium probability vector is an eigen vector of the transfer matrix associated to eigenvalue 1.

This observation allows us to compute the steady state value of the probability vector, as the eigenvector of the transfer matrix associated to the eigenvalue 1, as the following code shows.

In [ ]:
neighbor =  [[1, 3, 0, 0], [2, 4, 0, 1], [2, 5, 1, 2],
             [4, 6, 3, 0], [5, 7, 3, 1], [5, 8, 4, 2],
             [7, 6, 6, 3], [8, 7, 6, 4], [8, 8, 7, 5]]
transfer = numpy.zeros((9, 9))
for k in range(9):
    for neigh in range(4): transfer[neighbor[k][neigh], k] += 0.25
eigenvalues, eigenvectors = numpy.linalg.eig(transfer)

epsilon = 0.00001
print "Eigenvalues:"
print eigenvalues
print "steady state probability vector:"
for iter in range(9):
    if (abs(eigenvalues[iter] - 1.0) < epsilon):
        for i in range(9):
            print "%0.5f" % eigenvectors[i][iter]

The eigenvector is proportional to the expected equilibrium probabilities, as expected. We also note that $1.0$, the eigenvalue corresponding to the equilibrium, is the largest eigenvalue. We will now examine the role of the other eigenvalues. In order to do so, let us subtract the equilibrium value from the output of the previous exact evaluation of the probabilities for all values of $t$:

In [ ]:
neighbor =  [[1, 3, 0, 0], [2, 4, 0, 1], [2, 5, 1, 2],
             [4, 6, 3, 0], [5, 7, 3, 1], [5, 8, 4, 2],
             [7, 6, 6, 3], [8, 7, 6, 4], [8, 8, 7, 5]]
transfer = numpy.zeros((9, 9))

for k in range(9):
    for neigh in range(4):
        transfer[neighbor[k][neigh], k] += 0.25
position = numpy.zeros(9)
position[8] = 1.0

gather_0_values = []

for t in range(100):
    print t, '  ', ["%0.5f" % abs(i - 1.0 / 9.0) for i in position]
    gather_0_values.append(abs(position[0]- 1.0 / 9.0))
    position = numpy.dot(transfer, position)

We can analyze these data graphically. Doing so, it appears that the deviation from the equilibrium probability asymptotically converges towards $\lambda_1^t$, with $\lambda_1$ the second largest eigenvalue of the transfer matrix:

In [ ]:
fig, ax = plt.subplots(figsize = (10, 10), dpi=100, frameon=False)

plt.title("Deviation from the equilibrium probability at site 0 (semi-log scale)")
ax.plot(range(100), gather_0_values, label="deviation")
ax.plot(range(100), [0.75**t for t in range(100)], label  = r"$0.75^t$")

ax.legend()
ax.set_yscale('log')

Therefore, noticing that $\lambda_1^t = \exp(-t |\ln(\lambda1)|) = exp(-t / \tau)$ for $0 < \lambda_1 < 1.0$ , it appears that the deviation from equilibrium behaves asymptotically as $e^{(-t/\tau)}$ with $\tau = 1.0 / |\ln(\lambda1)| \simeq 3.476$. The second largest eigenvalue of the transfer matrix is therefore related to the characteristic time $\tau$ of the convergence.

You may observe that the value displayed by the code above confirm this expected behavior: for $t = \mathcal{O}(\tau)$, the system has reached the steady state, with the deviation from equilibrium being indistinguishable from 0. This can be explained simply by considering the following: it is possible to decompose $\pi_0^{t=0}$ on the set of eigenvectors of the transfer matrix, and observe its evolution in time like this:

$\pi_0^{t} = M^t \pi_0^{t=0} = P^{-1} D^t P \pi_0^{t=0} \Leftrightarrow (P \pi_0^{t}) =D^t (P \pi_0^{t=0})$, with $D$ the diagonal matrix holding the eigenvalues of the transfer matrix, and $P$ the matrix composed of the eigenvectors. As we see, when the Markov chain process is iterated, the projection of $\pi_0^{t=0}$ on the eigenvector associated to $\lambda=1$ is conserved, while all the others decay. The slowest decay is associated to the largest eigenvalue strictly smaller than 1.0, i.e. $\lambda_1$. This explains the behavior observed in the figure above.

## Irreducibility and aperiodicity in the Markov chain.

Let us come back to the decomposition written above:

$\pi^{t=0}  = \sum_i \lambda_i v_i$,

$\pi^{t}  = \sum_i \lambda_i^t v_i$,

with $v_i$ the eigenvectors of the transfer matrix. If any of the eigenvalues $\lambda_i$ has a modulus strictly larger than 1.0, then $\pi^t$ diverges, which is forbidden, considering that it represents a probability. Therefore, as we have seen above, at large times, the proability converges towards the steady state probability, which matches the eigenvector associated to the largest eigenvalue $\lambda_0 = 1$. We will now examine the mathematical conditions under which this is verified, and in particular, the necessary conditions for the steady state to be uniquely defined.

### 1$^{st}$ condition: irreducibility

Consider the system made up of two identical copies of the $3 \times 3$ lattice, inserted into a $6 \times 6$ frame, but still with only one pebble traveling around. If the chain starts in the upper left copy of the lattice, it will stay there, and the same will happen if it starts in the other lattice. The corresponding transfer matrix is the following:

$T = \dfrac{1}{4}
\begin{pmatrix}
2 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
1 & 1 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 2 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 1 & 0 & 0 & 2 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 1 & 0 & 1 & 1 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 2 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 2 & 1 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 1 & 1 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 2 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 1 & 0 & 0 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 2 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 1 & 1 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 1 & 2 \\
\end{pmatrix}$

The zeros in the upper right and lower left sectors correspond to the impossibility for the pebble to move from one sub lattice to the other sub lattice. Physically, we are faced with the unpleasant case, where the state of the system depends on the initial configuration, even at infinite times. Mathematically, this corresponds to the fact that the transfer matrix has two eigenvalues equal to 1.0. The situation can also be described by stating the the transfer matrix of the enlarged matrix is reducible - an undesirable feature for our study. This illustrates the following statement:

**In order for a Markov chain algorithm to be a valid Monte Carlo algorithm, its transfer matrix has to be irreducible.**

The following program shows how we can switch from a reducible (with two eigenvalues equal to 1.0) to an irreducible system, by allowing for a finite transition probability between the two sub-systems.

In [ ]:
neighbor =  [[1, 3, 0, 0], [2, 4, 0, 1], [2, 5, 1, 2],
             [4, 6, 3, 0], [5, 7, 3, 1], [5, 8, 4, 2],
             [7, 6, 6, 3], [8, 7, 6, 4], [8, 8, 7, 5]]
transfer = numpy.zeros((18, 18))

for k in range(9):
    for neigh in range(4): 
        transfer[neighbor[k][neigh], k] += 0.25     # upper left pebble game
        transfer[neighbor[k][neigh] + 9, k + 9] += 0.25 # lower right pebble game

# small transition epsilon between red 2 and blue 6
# epsilon = 0.04
epsilon = 0.0
transfer[6 + 9, 2] = transfer[2 , 6 + 9] = epsilon
transfer[2, 2] -= epsilon
transfer[6 + 9, 6 + 9] -= epsilon
eigenvalues, eigenvectors = numpy.linalg.eig(transfer)

eigenvalues.sort()
print eigenvalues
 
# you may print the eigenvectors by uncommenting the following four lines
#for iter in range(18):
#    print eigenvalues[iter]
#    for i in range(18):
#        print eigenvectors[i][iter]

The next version of the code produces a set of images illustrating the dynamics of the Markov chain. The probability for the chain to switch from one sub-system to the other is controlled by "epsilon".

In [ ]:
%matplotlib inline
random.seed('1234')
sigma = 0.4
epsilon = 0.4  # probability to switch from red to blue pebble, and vice versa
pylab.figure()
s_map_red =  [(1.0, 1.0), (2.0, 1.0), (3.0, 1.0), 
              (1.0, 2.0), (2.0, 2.0), (3.0, 2.0), 
              (1.0, 3.0), (2.0, 3.0), (3.0, 3.0)] 
offset = 3.0
s_map_blue = [(x + offset, y - offset) for (x,y) in s_map_red]
neighbor =  [[1, 3, 0, 0], [2, 4, 0, 1], [2, 5, 1, 2],
             [4, 6, 3, 0], [5, 7, 3, 1], [5, 8, 4, 2],
             [7, 6, 6, 3], [8, 7, 6, 4], [8, 8, 7, 5]]

color = 'red'  #chose 'red' or 'blue'
site = 8
tmax = 240
for iter in range(tmax):
    period = 4
    if (iter % period) == 0:
# Begin graphical output
        maxlength = len(str(tmax - 1))
        number_string = str(iter).zfill(maxlength)
        if color == 'red': 
            cir = pylab.Circle(s_map_red[site],  radius=sigma, fc='r')
        if color == 'blue':
            cir = pylab.Circle(s_map_blue[site], radius=sigma, fc='b')
        pylab.figure()
        pylab.gca().add_patch(cir)
        pylab.plot([0.5, 3.5], [0.5, 0.5], 'r')
        pylab.plot([0.5, 3.5], [1.5, 1.5], 'r')
        pylab.plot([0.5, 3.5], [2.5, 2.5], 'r')
        pylab.plot([1.5, 1.5], [0.5, 3.5], 'r')
        pylab.plot([2.5, 2.5], [0.5, 3.5], 'r')
        pylab.plot([3.5, 3.5], [0.5, 3.5], 'r')
        pylab.plot([0.5 + offset, 3.5 + offset], [1.5 - offset, 1.5 - offset], 'b')
        pylab.plot([0.5 + offset, 3.5 + offset], [2.5 - offset, 2.5 - offset], 'b')
        pylab.plot([0.5 + offset, 3.5 + offset], [3.5 - offset, 3.5 - offset], 'b')
        pylab.plot([0.5 + offset, 0.5 + offset], [0.5 - offset, 3.5 - offset], 'b')
        pylab.plot([1.5 + offset, 1.5 + offset], [0.5 - offset, 3.5 - offset], 'b')
        pylab.plot([2.5 + offset, 2.5 + offset], [0.5 - offset, 3.5 - offset], 'b')
        pylab.title('t = ' + number_string)
        pylab.axis('scaled')
        pylab.axis([0.5, 6.5, -2.5, 3.5])
        pylab.xticks([])
        pylab.yticks([])
        number_string_filename = str(iter / period).zfill(3)
        pylab.savefig('pebble_dual_movie_epsilon_' + number_string_filename + '.png', 
                      transparent=True)
        pylab.clf()
        pylab.close()
    # End of graphical output
    newsite = neighbor[site][ random.randint(0, 3)]
    newcolor = color
    if (color == 'red') and (site == 2) and (newsite == 2):
        if random.random() < epsilon:
            newcolor = 'blue'
            newsite = 6
            print "transition red->blue at time = ", iter
    if (color == 'blue') and (site == 6) and (newsite == 6):
        if random.random() < epsilon:
            newcolor = 'red'
            newsite = 2
            print "transition blue->red at time = ", iter
    site = newsite
    color = newcolor

### 2$^{nd}$ condition: aperiodicity

Let us imagine that one of the eigenvalues has modulus 1.0, while being different from 1.0, e.g. a complex eigenvalue. In such case, the projection of $\pi^t$ onto the corresponding eigenvalues never fades, and the $\pi^t$ vector oscillates, even in the limit of infinite times.

This is illustrated by a simple $2 \times 1$ lattice, in which the Markov chain has to jump to the unoccupied site at each step. The corresponding transfer matrix is:

\begin{pmatrix}
0 & 1 \\
1 & 0 
\end{pmatrix}

we have $\lambda_0 = 1.0$, $\lambda_1 = -1.0$, $v_0 = \begin{pmatrix} 1/2 \\ 1/2 \end{pmatrix}$, $v_1 = \begin{pmatrix} 1/2 \\ -1/2 \end{pmatrix}$, so that the probability vector does not converge for $t \to \infty$.

The generalization of this behavior corresponds to periodicity: in this simple setting, the system comes back to the same configuration every two time steps, with probability one. This introduces the second condition of validity for a Markov chain Monte Carlo algorithm: it should not be possible to determine a fixed finite number of steps after which the system comes back to the same configuration with probability 1.

Here again, it is possible to tweak the system slightly so that the periodicity vanishes. For instance we can introduce a small but finite probability for the pebble to remain in its current position, thereby eliminating the periodicity in the system. This is illustrated in the code snippet below.

In [ ]:
#epsilon = 0.01
epsilon = 0.0
transfer = [[ epsilon, 1.0 - epsilon ],
            [ 1.0 - epsilon, epsilon ]]
eigenvalues, eigenvectors = numpy.linalg.eig(transfer)
print eigenvalues
 
# you may print the eigenvectors by uncommenting the following lines...
#for iter in range(2):
#    print eigenvalues[iter]
#    for i in range(2):
#       print eigenvectors[i][iter]



The next version of the code produces a set of images illustrating the dynamics of the Markov chain. The probability for the chain to remain in the same spot is controlled by "epsilon".

In [ ]:
sigma = 0.4
epsilon = 0.1
pylab.figure()
s_map = [(1.0, 1.0), (2.0, 1.0)] 
neighbor =  [[1], [0]]
pos = 0
tmax = 20
for iter in range(tmax):
    # Begin of the graphics output
    pylab.figure()
    number_string = str(iter).zfill(len(str(tmax)))
    cir = pylab.Circle(s_map[pos], radius=sigma, fc='r')
    pylab.gca().add_patch(cir)
    pylab.plot([1.5, 1.5], [0.5, 1.5], 'b')
    pylab.title('t = '+ number_string)
    pylab.axis('scaled')
    pylab.axis([0.5, 2.5, 0.5, 1.5])
    pylab.xticks([])
    pylab.yticks([])
    pylab.savefig('2x1pebble_epsilon' + number_string + '.png',
                  transparent=True)
    pylab.close()
    # End of the graphics output
    newpos = neighbor[pos][0]
    if random.random() < epsilon:
        newpos = pos
    pos = newpos


As a conclusion, we have shown that two conditions have to be added to the one introduced in the lecture, and we end up with the three following conditions for a Markov chain Monte Carlo algorithm:

* global balance
* irreducibility
* aperiodicity

# Conclusion

We have shown that the Markov chain sampling is equivalent to a dynamics controlled by a transfer matrix. It to equilibrium in the limit $t \to \infty$, but exponentially fast, with a time scale given by the second largest eigenvalue of the transfer matrix. The concept of equilibrium, and the existence of a time scale associated with it, are one of the most precious features of Monte Carlo approaches. The precise evaluation of this time scale is often a challenge in Monte Carlo approaches, as will be discussed in the following sessions.